# CONTEXTO DO PROBLEMA

A **Insurance All** é uma empresa que fornece seguro de saúde para seus clientes e o time de produtos está analisando a possibilidade de oferecer aos assegurados, um novo produto: Um seguro de automóveis.

Assim como o seguro de saúde, os clientes desse novo plano de seguro de automóveis precisam pagar um valor anualmente à Insurance All para obter um valor assegurado pela empresa, destinado aos custos de um eventual acidente ou dano ao veículo.

A Insurance All fez uma pesquisa com cerca de 380 mil clientes sobre o interesse em aderir a um novo produto de seguro de automóveis, no ano passado. Todos os clientes demonstraram interesse ou não em adquirir o seguro de automóvel e essas respostas ficaram salvas em um banco de dados junto com outros atributos dos clientes.

O time de produtos selecionou 127 mil novos clientes que não responderam a pesquisa para participar de uma campanha, no qual receberão a oferta do novo produto de seguro de automóveis. A oferta será feita pelo time de vendas através de ligações telefônicas.

Contudo, o time de vendas tem uma capacidade de realizar 20 mil ligações dentro do período da campanha.

# O PROBLEMA

Nesse contexto, você foi contratado como um consultor de Ciência de Dados para construir um modelo que prediz se o cliente estaria ou não interessado no seguro de automóvel. 

Com a sua solução, o time de vendas espera conseguir priorizar as pessoas com maior interesse no novo produto e assim, otimizar a campanha realizando apenas contatos aos clientes mais propensos a realizar a compra.


Como resultado da sua consultoria, você precisará entregar um relatório contendo algumas análises e respostas às seguintes perguntas:

**1.** Principais Insights sobre os atributos mais relevantes de clientes interessados em adquirir um seguro de automóvel.

**2.** Qual a porcentagem de clientes interessados em adquirir um seguro de automóvel, o time de vendas conseguirá contatar fazendo 20.000 ligações?

**3.** E se a capacidade do time de vendas aumentar para 40.000 ligações, qual a porcentagem de clientes interessados em adquirir um seguro de automóvel o time de vendas conseguirá contatar?

**4.** Quantas ligações o time de vendas precisa fazer para contatar 80% dos clientes interessados em adquirir um seguro de automóvel?

# ENTENDIMENTO DO PROBLEMA

- **Motivação:**
    - A empresa vai realizar uma campanha para um novo produto (seguro de automóvel).


- **Causa Raiz do Problema:**
    - Identificar clientes interessados no seguro do automóvel.


- **Dados para Resolver o Problema:**
    - Os dados estão armazenados em um banco de dados PostgresSQL na AWS.
    

- **Planejamento da Solução:**

# 0.0 IMPORTS

In [58]:
import psycopg2 as pg
import boto3
import pandas as pd

## 0.1 Funções Auxiliares

## 0.2 Coleta de Dados

In [64]:
# Creadentials
HOST= 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
PORT='5432'
USER='member'
DATABASE='comunidadedsdb'
PASSWORD = 'cdspa'


try: # Connect DB
    conn = pg.connect(
        host=HOST,port=PORT, database=DATABASE, user=USER, password=PASSWORD
    )
    
    # Query data
    query = """
        SELECT *
        FROM pa004.users u LEFT JOIN pa004.vehicle v on (u.id = v.id)
                           LEFT JOIN pa004.insurance i on (u.id = i.id)  
        WHERE u.age > 44
"""
    
    # Read in sql and parse to Dataframe
    df = pd.read_sql(query, conn)
    df = df.drop(df.columns[[5, 9]], axis=1)
    
except Exception as e: # Connection Error  
    print("Database connection failed due to {}".format(e))
    
df.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Female,56,28.0,26.0,1,1-2 Year,Yes,0,32031.0,72,1
1,Female,47,35.0,124.0,1,1-2 Year,Yes,0,47576.0,46,1
2,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
3,Female,54,28.0,122.0,1,1-2 Year,No,0,38560.0,184,0
4,Female,49,3.0,152.0,1,1-2 Year,No,1,25553.0,130,0


# 1.0 DESCRIÇÃO DE DADOS